In [3]:
# Import necessary packages
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

In [4]:
# get url
url = 'https://www.zalando.de/sportschuhe-herren/?p=1'

PATH = r"C:\Program Files (x86)\msedgedriver.exe"

driver = webdriver.Edge(PATH)
driver.implicitly_wait(30)

driver.get(url)

html = driver.page_source
p_soup = BeautifulSoup(html, 'lxml')

print(p_soup.prettify())

<html class="js-focus-visible z-base-assets-major-5 z-base-assets-minor-3 z-base-assets-patch-1" data-js-focus-visible="" lang="de-DE">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <script async="" src="https://www.google-analytics.com/plugins/ua/ec.js" type="text/javascript">
  </script>
  <script crossorigin="" src="https://mosaic01.ztat.net/base-assets/require-2.1.22.min.js">
  </script>
  <script>
   var TailorPipe=(function(n,e){function t(){for(var n,e=c.length-1;e>=0;e--)if((n=c[e]).hasAttribute("data-pipe"))return n.removeAttribute("data-pipe"),n}function o(){0!==l.length||!n.readyState||"complete"!==n.readyState&&"interactive"!==n.readyState||v.onDone()}function r(n){return null!=n&&"object"==typeof n&&"function"==typeof n.then}function i(n,e,t,i){v.onBeforeInit(t,i);var a=n(e),f=function(){l.pop(),v.onAfterInit(t,i),o(t)};r(a)?a.then(f).catch(f):f()}function a(n,t){!1 in e||p.push({name:n,duration:Number(t),entryType:"tailor",st

In [8]:
shoes = p_soup.find_all('div', class_='qMZa55 SQGpu8 iOzucJ JT3_zV DvypSJ')

# Function to get the term containing 'schuh'
'''def get_term(doc_list):
    for word in doc_list:
        if 'schuh' in word:
            return word.replace('/','')'''

# Function to get the first item of list
def first_item(list):
    if len(list)<=2:
        return list[0]

# Function to get the second item of list
def second_item(list):
    if len(list)==2:
        return list[1]

# Create the list of shoes
shoes_list = []

for shoe in shoes:
    # create a dictionary
    shoes_dict = {}
    
    shoes_dict['brand_name'] = shoe.find('span', class_='u-6V88 ka2E9k uMhVZi FxZV-M uc9Eq5 pVrzNP ZkIJC- r9BRio qXofat EKabf7').text
    shoes_dict['shoe_type'] = shoe.find('h3', class_='u-6V88 ka2E9k uMhVZi FxZV-M z-oVg8 pVrzNP ZkIJC- r9BRio qXofat EKabf7').text.split('-')[:2]
    #shoes_dict['shoe_type'] = get_term( shoes_dict['shoe_type'])
    
    price = shoe.find('div', class_='_0xLoFW u9KIT8 _7ckuOK').text.replace('ab','').replace('€', '')
    shoes_dict['original_price'] = first_item(price.split())
    shoes_dict['sale_price'] = second_item(price.split())
    
    try:
        weight = shoe.find('span','VnVJx_ ka2E9k uMhVZi FxZV-M z-oVg8 weHhRC').text
        shoes_dict['shoe_weight'] = weight.replace('g', '')
    except Exception as e:
        shoes_dict['shoe_weight'] = 'unknown'
    
    shoes_list.append(shoes_dict)
#print(shoes_list)
   
    #print(brand_name)
    #print(find(shoe_type))
    #print(shoe_type) 
    #print(shoe_weight)
    #print(first_item(original_price))
    #print(second_item(sale_price))

In [27]:
# Create dataframe
shoes = pd.DataFrame(shoes_list)
shoes

,brand_name,shoe_type,original_price,sale_price,shoe_weight
0,The North Face,Laufschuh Trail,"147,00",None,unknown
1,The North Face,None,"35,00",None,unknown
2,The North Face,Hikingschuh,"135,95",None,unknown
3,Nike Performance,Laufschuh Neutral,"55,00","49,90",304
4,adidas Performance,Laufschuh Neutral,"48,55",None,244
...,...,...,...,...,...
79,adidas Performance,Laufschuh Neutral,"48,55",None,263
80,Under Armour,Laufschuh Neutral,"72,95","54,45",302
81,ASICS,Laufschuh Trail,"59,95","49,90",264
82,ASICS,Laufschuh Neutral,"140,00","99,90",unknown


### Web Scraping from multiple pages

In [20]:
# Create the list of shoes
shoes_list1 = []

# Scrape data from first 3 pages
for page_num in range(1,4):

    PATH = r"C:\Program Files (x86)\msedgedriver.exe"

    driver = webdriver.Edge(PATH)
    driver.get('https://www.zalando.de/sportschuhe-herren/?p={}'.format(page_num))
    time.sleep(5)
    #driver.quit()
    #print(pages)

    html = driver.page_source
    p_soup = BeautifulSoup(html, 'lxml')
    
    shoes = p_soup.find_all('div', class_='qMZa55 SQGpu8 iOzucJ JT3_zV DvypSJ')
    
    for shoe in shoes:
        driver.implicitly_wait(30)
        
        # create a dictionary
        shoes_dict1 = {}

        shoes_dict1['brand_name'] = shoe.find('span', class_='u-6V88 ka2E9k uMhVZi FxZV-M uc9Eq5 pVrzNP ZkIJC- r9BRio qXofat EKabf7').text
        shoes_dict1['shoe_type'] = shoe.find('h3', class_='u-6V88 ka2E9k uMhVZi FxZV-M z-oVg8 pVrzNP ZkIJC- r9BRio qXofat EKabf7').text.split('-')[:2]

        price = shoe.find('div', class_='_0xLoFW u9KIT8 _7ckuOK').text.replace('ab','').replace('€', '')
        shoes_dict1['original_price'] = first_item(price.split())
        shoes_dict1['sale_price'] = second_item(price.split())

        try:
            weight = shoe.find('span','VnVJx_ ka2E9k uMhVZi FxZV-M z-oVg8 weHhRC').text
            shoes_dict1['shoe_weight'] = weight.replace('g', '')
        except Exception as e:
            shoes_dict1['shoe_weight'] = 'unknown'

        shoes_list1.append(shoes_dict1)

print(shoes_list1)


In [19]:
# Create dataframe and save as a csv file
#shoes1 = pd.DataFrame(shoes_list1)
#shoes1.to_csv('shoes_data.csv')

In [24]:
shoes_data = pd.read_csv('shoes_data.csv', index_col=0)
shoes_data.shape

(252, 5)

In [25]:
shoes_data.head()

,brand_name,shoe_type,original_price,sale_price,shoe_weight
0,adidas Performance,"['RUNFALCON 2.0 TR ', ' Laufschuh Neutral ']","59,95",NaN,310
1,Mammut,"['HUECO ', ' Hikingschuh ']","150,00",NaN,unknown
2,Mammut,"['ALVRA ', ' Hikingschuh ']","160,00",NaN,unknown
3,adidas Performance,['ULTRABOOST 20 PRIMEBLUE PRIMEKNIT RUNNING SH...,"107,95","179,95",328
4,Nike Performance,"['REVOLUTION 5 ', ' Laufschuh Neutral ']","54,95",NaN,299
